## Manipulation Methods

### where , select , apply , if then else 

.apply method execute for every value in series , so avoid it 

In [52]:
import pandas as pd 
url = 'https://github.com/arunadas/effective-pandas/raw/main/data/vehicles.csv.zip'
df = pd.read_csv(url,dtype='unicode')
city_mpg = df.city08.astype(int)
highway_mpg = df.highway08.astype(int)
make = df.make

In [6]:
def gt20(val):
    return val > 20 

In [13]:
%%timeit
city_mpg.apply(gt20)

2.51 ms ± 17.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [12]:
#broadcast method 
%timeit city_mpg.gt(20)

18 µs ± 109 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [15]:
make.size

41144

In [17]:
make.sample(n=3)

5483       Toyota
13671         GMC
11298    Chrysler
Name: make, dtype: object

In [18]:
make.value_counts()

make
Chevrolet                      4003
Ford                           3371
Dodge                          2583
GMC                            2494
Toyota                         2071
                               ... 
Volga Associated Automobile       1
Panos                             1
Mahindra                          1
Excalibur Autos                   1
London Coach Co Inc               1
Name: count, Length: 136, dtype: int64

In [20]:
top5 = make.value_counts().index[:5]
def generalize_top5(val):
    if val in top5:
        return val
    return 'other'    

In [24]:
# apply will call function generalize_top5 for each value 
%timeit make.apply(generalize_top5)

12.2 ms ± 133 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [25]:
%timeit make.where(make.isin(top5), other='Other')

1.17 ms ± 10.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [26]:
make.where(make.isin(top5), other='Other')

0        Other
1        Other
2        Dodge
3        Dodge
4        Other
         ...  
41139    Other
41140    Other
41141    Other
41142    Other
41143    Other
Name: make, Length: 41144, dtype: object

In [27]:
#mask is complement of where where ever condition is false it keeps the original value
make.mask(make.isin(top5), other='Other')

0        Alfa Romeo
1           Ferrari
2             Other
3             Other
4            Subaru
            ...    
41139        Subaru
41140        Subaru
41141        Subaru
41142        Subaru
41143        Subaru
Name: make, Length: 41144, dtype: object

In [29]:
#~ inversion of boolean array switching all true to false or vice versa
make.mask(~make.isin(top5), other='Other')

0        Other
1        Other
2        Dodge
3        Dodge
4        Other
         ...  
41139    Other
41140    Other
41141    Other
41142    Other
41143    Other
Name: make, Length: 41144, dtype: object

In [31]:
vc = make.value_counts()
top5 = vc.index[:5]
top10 = vc.index[:10]

def generalize(val):
    if val in top5:
       return val
    elif val in top10:
       return 'Top10'
    else:
        return 'Other'

In [32]:
make.apply(generalize)

0        Other
1        Other
2        Dodge
3        Dodge
4        Other
         ...  
41139    Other
41140    Other
41141    Other
41142    Other
41143    Other
Name: make, Length: 41144, dtype: object

In [34]:
# usage of pandas
(make
.where(make.isin(top5), 'Top10')
.where(make.isin(top10), 'Other'))

0        Other
1        Other
2        Dodge
3        Dodge
4        Other
         ...  
41139    Other
41140    Other
41141    Other
41142    Other
41143    Other
Name: make, Length: 41144, dtype: object

In [35]:
# another option is select from numpy
import numpy as np
np.select([make.isin(top5), make.isin(top10)],
          [make,'Top10'], 'Other')


array(['Other', 'Other', 'Dodge', ..., 'Other', 'Other', 'Other'],
      dtype=object)

np.select(condlist, choicelist, default):

condlist: A list of boolean conditions.
choicelist: A list of values corresponding to each condition in condlist.
default: A fallback value used when none of the conditions are True

output of select is numpy array you can wrap it in series if you want series


In [36]:
pd.Series(np.select([make.isin(top5), make.isin(top10)],
          [make,'Top10'], 'Other'))

0        Other
1        Other
2        Dodge
3        Dodge
4        Other
         ...  
41139    Other
41140    Other
41141    Other
41142    Other
41143    Other
Length: 41144, dtype: object

### Missing Data

In [37]:
cyl = df.cylinders

In [39]:
(cyl.isna().sum())

np.int64(206)

In [40]:
missing = cyl.isna()

In [41]:
make.loc[missing]

7138     Nissan
7139     Toyota
8143     Toyota
8144       Ford
8146       Ford
          ...  
34563     Tesla
34564     Tesla
34565     Tesla
34566     Tesla
34567     Tesla
Name: make, Length: 206, dtype: object

### Filling in missing Data 

In [43]:
cyl[cyl.isna()]

7138     NaN
7139     NaN
8143     NaN
8144     NaN
8146     NaN
        ... 
34563    NaN
34564    NaN
34565    NaN
34566    NaN
34567    NaN
Name: cylinders, Length: 206, dtype: object

In [45]:
cyl.fillna(0).loc[7136:7141]
# no mutation of data occurs in any operations

7136    6
7137    6
7138    0
7139    0
7140    6
7141    6
Name: cylinders, dtype: object

### Interpolating Data

In [46]:
temp = pd.Series([32, 40, None , 42, 39, 32])

In [47]:
temp.interpolate()

0    32.0
1    40.0
2    41.0
3    42.0
4    39.0
5    32.0
dtype: float64

### Clipping Data

In [49]:
# outliners from the dataset clip uses lower and upper bound to replace the values
city_mpg.loc[:446]

0      19
1       9
2      23
3      10
4      17
       ..
442    15
443    15
444    15
445    15
446    31
Name: city08, Length: 447, dtype: int64

In [51]:
(city_mpg.loc[:446]
      .clip(lower=city_mpg.quantile(0.05),
            upper=city_mpg.quantile(0.95)))

0      19
1      11
2      23
3      11
4      17
       ..
442    15
443    15
444    15
445    15
446    27
Name: city08, Length: 447, dtype: int64

### Sorting Values

In [53]:
city_mpg.sort_values()

7901       6
34557      6
37161      6
21060      6
35887      6
        ... 
34563    138
34564    140
32599    150
31256    150
33423    150
Name: city08, Length: 41144, dtype: int64

In [54]:
# also perfor, math operation on sorted series
(city_mpg.sort_values() + highway_mpg) / 2

0        22.0
1        11.5
2        28.0
3        11.0
4        20.0
         ... 
41139    22.5
41140    24.0
41141    21.0
41142    21.0
41143    18.5
Length: 41144, dtype: float64

### Sorting the index

In [56]:
# reverse the index sorted values
city_mpg.sort_values().sort_index()

0        19
1         9
2        23
3        10
4        17
         ..
41139    19
41140    20
41141    18
41142    18
41143    16
Name: city08, Length: 41144, dtype: int64

### Drop duplicates

In [57]:
# Remove duplicates you can choose to keep first or last , default is first
city_mpg.drop_duplicates()

0         19
1          9
2         23
3         10
4         17
        ... 
34364    127
34409    114
34564    140
34565    115
34566    104
Name: city08, Length: 105, dtype: int64

In [59]:
test = pd.Series([40,20,30,20,10])
test.drop_duplicates()

0    40
1    20
2    30
4    10
dtype: int64

In [60]:
test.drop_duplicates(keep="last")

0    40
2    30
3    20
4    10
dtype: int64

In [62]:
test.drop_duplicates(keep=False)

0    40
2    30
4    10
dtype: int64

### Ranking Data

In [63]:
# rank method can have min and dense 
# default is average of values you can redfine as min or dense
city_mpg.rank()

0        27060.5
1          235.5
2        35830.0
3          607.5
4        19484.0
          ...   
41139    27060.5
41140    29719.5
41141    23528.0
41142    23528.0
41143    15479.0
Name: city08, Length: 41144, dtype: float64

In [64]:
city_mpg.rank(method='min')

0        25555.0
1          136.0
2        35119.0
3          336.0
4        17467.0
          ...   
41139    25555.0
41140    28567.0
41141    21502.0
41142    21502.0
41143    13492.0
Name: city08, Length: 41144, dtype: float64

In [65]:
city_mpg.rank(method='dense')

0        14.0
1         4.0
2        18.0
3         5.0
4        12.0
         ... 
41139    14.0
41140    15.0
41141    13.0
41142    13.0
41143    11.0
Name: city08, Length: 41144, dtype: float64

In [66]:
test

0    40
1    20
2    30
3    20
4    10
dtype: int64

In [67]:
test.rank(method='min')

0    5.0
1    2.0
2    4.0
3    2.0
4    1.0
dtype: float64

In [68]:
test.rank(method='dense')

0    4.0
1    2.0
2    3.0
3    2.0
4    1.0
dtype: float64

### Replace

In [69]:
make.replace('Subaru', 'スバル')

0        Alfa Romeo
1           Ferrari
2             Dodge
3             Dodge
4               スバル
            ...    
41139           スバル
41140           スバル
41141           スバル
41142           スバル
41143           スバル
Name: make, Length: 41144, dtype: object

In [70]:
# you can use regex \1 for first parenthesis
make.replace(r'(Fer)ra(r.*)',value=r'\2-other-\1',regex=True)

0          Alfa Romeo
1        ri-other-Fer
2               Dodge
3               Dodge
4              Subaru
             ...     
41139          Subaru
41140          Subaru
41141          Subaru
41142          Subaru
41143          Subaru
Name: make, Length: 41144, dtype: object

In [71]:
rep_test = pd.Series(['Dave','Suzy','Adam','Liv'])

In [72]:
rep_test

0    Dave
1    Suzy
2    Adam
3     Liv
dtype: object

In [73]:
rep_test.replace(to_replace='Suzy', value='Suzanne')

0       Dave
1    Suzanne
2       Adam
3        Liv
dtype: object

In [74]:
rep_test.replace(to_replace={'Suzy':'Suzanne'})

0       Dave
1    Suzanne
2       Adam
3        Liv
dtype: object

In [75]:
rep_test.replace(to_replace='z.*',value = 'zanne', regex=True)

0       Dave
1    Suzanne
2       Adam
3        Liv
dtype: object

### Binning Data 

In [77]:
# create the category dataset you can define the bins
pd.cut(city_mpg, 10)

0        (5.856, 20.4]
1        (5.856, 20.4]
2         (20.4, 34.8]
3        (5.856, 20.4]
4        (5.856, 20.4]
             ...      
41139    (5.856, 20.4]
41140    (5.856, 20.4]
41141    (5.856, 20.4]
41142    (5.856, 20.4]
41143    (5.856, 20.4]
Name: city08, Length: 41144, dtype: category
Categories (10, interval[float64, right]): [(5.856, 20.4] < (20.4, 34.8] < (34.8, 49.2] < (49.2, 63.6] ... (92.4, 106.8] < (106.8, 121.2] < (121.2, 135.6] < (135.6, 150.0]]

In [79]:
# they are half-open intervals values outside the bins category are included
pd.cut(city_mpg, [0,10,20,40,70,150])

0        (10, 20]
1         (0, 10]
2        (20, 40]
3         (0, 10]
4        (10, 20]
           ...   
41139    (10, 20]
41140    (10, 20]
41141    (10, 20]
41142    (10, 20]
41143    (10, 20]
Name: city08, Length: 41144, dtype: category
Categories (5, interval[int64, right]): [(0, 10] < (10, 20] < (20, 40] < (40, 70] < (70, 150]]

In [80]:
pd.qcut(city_mpg, 10)

0         (18.0, 20.0]
1        (5.999, 13.0]
2         (21.0, 24.0]
3        (5.999, 13.0]
4         (16.0, 17.0]
             ...      
41139     (18.0, 20.0]
41140     (18.0, 20.0]
41141     (17.0, 18.0]
41142     (17.0, 18.0]
41143     (15.0, 16.0]
Name: city08, Length: 41144, dtype: category
Categories (10, interval[float64, right]): [(5.999, 13.0] < (13.0, 14.0] < (14.0, 15.0] < (15.0, 16.0] ... (18.0, 20.0] < (20.0, 21.0] < (21.0, 24.0] < (24.0, 150.0]]

In [81]:
pd.qcut(city_mpg, 10, labels=list(range(1,11)))

0        7
1        1
2        9
3        1
4        5
        ..
41139    7
41140    7
41141    6
41142    6
41143    4
Name: city08, Length: 41144, dtype: category
Categories (10, int64): [1 < 2 < 3 < 4 ... 7 < 8 < 9 < 10]

### Exercise 9.14 


In [104]:
#1. Create a series from a numeric column that has the value of 'high' if it is equal to or above the mean and 
#'low' if it is below the mean using .apply

numeric = pd.Series([2,3,1,5,3,2])
mean = numeric.mean()
# print(mean)
# print(numeric)
def genelaize(val):
    if val >= mean:
        return 'high'
    else:
        return 'low'
numeric.apply(genelaize)

0     low
1    high
2     low
3    high
4    high
5     low
dtype: object

In [99]:
#2. Create a series from a numeric column that has the value of 'high' if it is equal to or above the mean and 
#'low' if it is below the mean using .select
# low = numeric.lt(mean)
# print(low)
import numpy as np
pd.Series(np.select(
    [numeric >= mean, numeric < mean],  # Conditions
    ['high', 'low'],                    # Choices
    default='unknown'                  
))


0     low
1    high
2     low
3    high
4    high
5     low
dtype: object

In [105]:
#3 timeit above 2 to see which one is faster
%timeit numeric.apply(genelaize)

15.8 µs ± 91.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [111]:
%timeit (pd.Series(np.select([numeric >= mean, numeric < mean],   ['high', 'low'], default='unknown')))

57.8 µs ± 444 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [119]:
#4 Replace the missing values of a numeric series with the median value
test = pd.Series([4,5,6,2,None,3,1])
median = test.median()
print(median)
test=test.fillna(median)

3.5


In [120]:
#5 Clip the values of a numeric series to between to 10th and 90th percentiles
(test
      .clip(lower=test.quantile(0.01),
            upper=test.quantile(0.90)))

0    4.00
1    5.00
2    5.40
3    2.00
4    3.50
5    3.00
6    1.06
dtype: float64

In [131]:
#6 Using a categorical column, replace any value that is not in the top 5 most frequent values with 'other'
category_series = pd.Series(['apple', 'banana','tomato','tomato', 'apple', 'orange', 'grape', 'apple', 
                             'banana', 'kiwi', 'banana','potato', 'apple','onion','onion','gwava','gwava','cherry','cherry',
                             'kiwi', 'pear','grape', 'pear','bindi','karela'])
top5 = category_series.value_counts().index[:5]
top10 = category_series.value_counts().index[:10]
print(top5)
result = category_series.where(category_series.isin(top5), 'other')
print(result)

Index(['apple', 'banana', 'tomato', 'grape', 'kiwi'], dtype='object')
0      apple
1     banana
2     tomato
3     tomato
4      apple
5      other
6      grape
7      apple
8     banana
9       kiwi
10    banana
11     other
12     apple
13     other
14     other
15     other
16     other
17     other
18     other
19      kiwi
20     other
21     grape
22     other
23     other
24     other
dtype: object


In [134]:
#7 Using a categorical column, replace any value that is not in the top 10 most frequent values with 'other'
print(top10)
result = category_series.where(category_series.isin(top10),'other')
print(result)

Index(['apple', 'banana', 'tomato', 'grape', 'kiwi', 'onion', 'gwava',
       'cherry', 'pear', 'orange'],
      dtype='object')
0      apple
1     banana
2     tomato
3     tomato
4      apple
5     orange
6      grape
7      apple
8     banana
9       kiwi
10    banana
11     other
12     apple
13     onion
14     onion
15     gwava
16     gwava
17    cherry
18    cherry
19      kiwi
20      pear
21     grape
22      pear
23     other
24     other
dtype: object


In [138]:
#8 Make a function that takes a categorical series and a number(n) and returns a replace series that replaces 
#any value that is not in the top n most frequent values with 'other'
S = category_series

def gen(S, n):
    # Get the top n most frequent values
    topn = S.value_counts().index[:n]
    # Replace values not in the top n with 'Other'
    result = S.where(S.isin(topn), 'Other')
    return result
n = 2    
result = gen(S,n)
print(result)

0      apple
1     banana
2      Other
3      Other
4      apple
5      Other
6      Other
7      apple
8     banana
9      Other
10    banana
11     Other
12     apple
13     Other
14     Other
15     Other
16     Other
17     Other
18     Other
19     Other
20     Other
21     Other
22     Other
23     Other
24     Other
dtype: object
